In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

Using TensorFlow backend.


In [2]:
y_train = pd.read_csv('../model_input/train_y.csv', index_col=0).values
y_valid = pd.read_csv('../model_input/valid_y.csv', index_col=0).values
y_test = pd.read_csv('../model_input/test_y.csv', index_col=0).values

In [3]:
X_train = np.concatenate((np.load('../model_input/train_pitch_x.npz')['arr_0'].reshape(-1, 300, 12),
                          np.load('../model_input/train_timbre_x.npz')['arr_0'].reshape(-1, 300, 12),
                          np.load('../model_input/train_loudness_x.npz')['arr_0'].reshape(-1, 300, 3)), axis=2).reshape(-1, 8100)
X_test = np.concatenate((np.load('../model_input/test_pitch_x.npz')['arr_0'].reshape(-1, 300, 12),
                         np.load('../model_input/test_timbre_x.npz')['arr_0'].reshape(-1, 300, 12),
                         np.load('../model_input/test_loudness_x.npz')['arr_0'].reshape(-1, 300, 3)), axis=2).reshape(-1, 8100)
X_valid = np.concatenate((np.load('../model_input/valid_pitch_x.npz')['arr_0'].reshape(-1, 300, 12),
                          np.load('../model_input/valid_timbre_x.npz')['arr_0'].reshape(-1, 300, 12),
                          np.load('../model_input/valid_loudness_x.npz')['arr_0'].reshape(-1, 300, 3)), axis=2).reshape(-1, 8100)

In [4]:
METRICS = [
    keras.metrics.MeanSquaredError(name='mse')
]

In [5]:
def build_model():
    model = keras.Sequential([
    Dense(2048, activation='relu', input_shape=(8100,)),
    #Dropout(0.5),
    Dense(64, activation='relu'),
    #Dropout(0.5),
    Dense(2)
  ])
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=METRICS)
    return model

In [6]:
model = build_model()

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2048)              16590848  
_________________________________________________________________
dense_2 (Dense)              (None, 64)                131136    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 16,722,114
Trainable params: 16,722,114
Non-trainable params: 0
_________________________________________________________________


In [8]:
es = keras.callbacks.EarlyStopping(monitor='val_mse', mode='min', verbose=1, patience=10, restore_best_weights=True)
ckpt = keras.callbacks.ModelCheckpoint(filepath='weights.hdf5', verbose=1, save_best_only=True, monitor='val_min',
                                       mode='min')

In [9]:
X_valid.shape

(3811, 8100)

In [10]:
y_valid.shape

(3811, 2)

In [11]:
EPOCHS = 1000
BATCH_SIZE = 64

history = model.fit(
  X_train, y_train, batch_size=BATCH_SIZE,
    epochs=EPOCHS, 
    validation_data=(X_valid, y_valid),
    verbose=1,
    callbacks=[
        #tfdocs.modeling.EpochDots(), 
        es])

Train on 11107 samples, validate on 3811 samples
Epoch 1/1000
11107/11107 [==============================] - 7s 640us/step - loss: 277.1540 - mse: 277.1541 - val_loss: 1.8483 - val_mse: 1.8483
Epoch 2/1000
11107/11107 [==============================] - 7s 598us/step - loss: 1.2488 - mse: 1.2488 - val_loss: 1.8516 - val_mse: 1.8516
Epoch 3/1000
11107/11107 [==============================] - 7s 597us/step - loss: 1.0498 - mse: 1.0498 - val_loss: 1.9294 - val_mse: 1.9294
Epoch 4/1000
11107/11107 [==============================] - 7s 597us/step - loss: 1.0485 - mse: 1.0485 - val_loss: 1.9280 - val_mse: 1.9280
Epoch 5/1000
11107/11107 [==============================] - 7s 597us/step - loss: 1.0471 - mse: 1.0471 - val_loss: 1.9264 - val_mse: 1.9264
Epoch 6/1000
11107/11107 [==============================] - 7s 600us/step - loss: 1.0456 - mse: 1.0456 - val_loss: 1.9248 - val_mse: 1.9248
Epoch 7/1000
11107/11107 [==============================] - 7s 598us/step - loss: 1.0441 - mse: 1.0441 - va